In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix)
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, LSTM, GRU, SimpleRNN, Flatten, Dropout, Reshape
from keras.optimizers import Adam



In [33]:

# Load dataset
df = pd.read_csv(r"C:\Users\user\Desktop\Project\EXPERIMENTS\revised-data\audio_features_cp.csv")

# Feature selection
features = [
    'mfcc_1', 'mfcc_2', 'mfcc_3', 'mfcc_4', 'mfcc_5', 'mfcc_6', 'mfcc_7', 'mfcc_8', 
    'mfcc_9', 'mfcc_10', 'mfcc_11', 'mfcc_12', 'mfcc_13', 'spectral_centroid', 
    'spectral_bandwidth', 'zero_crossing_rate', 'spectrogram_mean', 
    'spectrogram_median', 'spectrogram_variance'
]
label = 'label'

# Feature Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df[features] = scaler.fit_transform(df[features])

# Split data
X = df[features].values
y = df[label].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshaping for each model
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

X_train_cnn1d = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test_cnn1d = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

X_train_cnn2d = X_train.reshape(X_train.shape[0], X_train.shape[1], 1, 1)
X_test_cnn2d = X_test.reshape(X_test.shape[0], X_test.shape[1], 1, 1)

# Define models
def create_rnn_model(input_shape):
    model = Sequential([
        SimpleRNN(128, activation='relu', input_shape=input_shape, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_gru_model(input_shape):
    model = Sequential([
        GRU(128, activation='relu', input_shape=input_shape, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_lstm_model(input_shape):
    model = Sequential([
        LSTM(128, activation='relu', input_shape=input_shape, return_sequences=False),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

def create_cnn1d_model(input_shape):
    model = Sequential([
        Conv1D(64, 3, activation='relu', input_shape=input_shape),
        MaxPooling1D(2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    return model

# Training and evaluation function
def train_and_evaluate_hybrid_model(n_iterations):
    evaluation = pd.DataFrame(columns=["Iteration", "Model", "Accuracy", "Precision", "Recall", "F1-Score"])
    
    for i in range(n_iterations):
        print(f"Iteration {i + 1}/{n_iterations}")

        # Train each model
        rnn_model = create_rnn_model(X_train_rnn.shape[1:])
        rnn_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy','precision','recall'])
        rnn_model.fit(X_train_rnn, y_train, epochs=250, verbose=0)
        rnn_predictions = (rnn_model.predict(X_test_rnn) > 0.5).astype(int)

        gru_model = create_gru_model(X_train_rnn.shape[1:])
        gru_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy','precision','recall'])
        gru_model.fit(X_train_rnn, y_train, epochs=250, verbose=0)
        gru_predictions = (gru_model.predict(X_test_rnn) > 0.5).astype(int)

        lstm_model = create_lstm_model(X_train_rnn.shape[1:])
        lstm_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy','precision','recall'])
        lstm_model.fit(X_train_rnn, y_train, epochs=250, verbose=0)
        lstm_predictions = (lstm_model.predict(X_test_rnn) > 0.5).astype(int)

        cnn1d_model = create_cnn1d_model(X_train_cnn1d.shape[1:])
        cnn1d_model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy','precision','recall'])
        cnn1d_model.fit(X_train_cnn1d, y_train, epochs=250, verbose=0)
        cnn1d_predictions = (cnn1d_model.predict(X_test_cnn1d) > 0.5).astype(int)

        # Hybrid predictions using majority voting
        hybrid_predictions = np.round(((2*rnn_predictions) + (10*gru_predictions) + (2*lstm_predictions) + (8*cnn1d_predictions))/22).astype(int)

        # Evaluation metrics
        accuracy = accuracy_score(y_test, hybrid_predictions)
        precision = precision_score(y_test, hybrid_predictions, zero_division=1)
        recall = recall_score(y_test, hybrid_predictions, zero_division=1)
        f1 = f1_score(y_test, hybrid_predictions, zero_division=1)

        evaluation = evaluation._append({
            "Iteration": i + 1,
            "Model": "Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)",
            "Accuracy": accuracy,
            "Precision": precision,
            "Recall": recall,
            "F1-Score": f1
        }, ignore_index=True)

    # Save results
    return evaluation


In [35]:
# Example usage
n_iterations = 100
results = train_and_evaluate_hybrid_model(n_iterations)
results.to_csv("hybrid_model_results.csv", index=False)
print(results)

Iteration 1/100


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step  
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step  
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 2/100


...

21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Iteration 100/100


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step 
21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step  


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
   Iteration                                        Model  Accuracy  \
0          1  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.948485   
1          2  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.945455   
2          3  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.953030   
3          4  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.946970   
4          5  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.950000   
..       ...                                          ...       ...   
95        96  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.951515   
96        97  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.950000   
97        98  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.950000   
98        99  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.946970   
99       100  Hybrid (RNN + GRU + LSTM + CNN 1D + CNN 2D)  0.945455   

    Precision    Recall  F1-Score  
0    0.948276  0.953757  0.951009  
1    0.950581  0.945087  0.947826  